# Automatically add article/journal info to a Google Scholar profile
Export your Google Scholar profile as CSV,

upload the file(s) from [here](./),

This program will add new columns like:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Institution Authors</th>
      <th>Institution Groups</th>
      <th>DOI</th>
      <th>ISSN</th>
      <th>Impact Factor</th>
      <th>Quartil</th>
      <th>Journal Hindex</th>
      <th>...</th>
    </tr>
  </thead>
</table>
The enriched CSV file can be saved and used as a [Spreadsheet](https://docs.google.com/spreadsheets/d/19dx09hBthKknZs1XxIKuGgwq3ieyPGx8Xr3hi4xqvBc/edit?usp=sharing).

Full Code in this public [repository](https://github.com/restrepo/articles).

To run the program use `<Shift>+<Enter>` in each Code Cell (or from the Menu: `Cell -> Run Cells`)

In [8]:
from publications import *

Here we assume that there exist a data base with the authors multiple names associated to the real name in the format:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Author_Names</th>
      <th>Control</th>
      <th>Full_Name</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td> Zapata, Oscar;Zapata, O.</td>
      <td> 0</td>
      <td> Óscar Alberto Zapata Noreña</td>
    </tr>
  </tbody>
</table>
Where control is some identification number associated to the author.


We assume also that there is a data base with the Group at which some real name author belongs in the format:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Full_Name</th>
      <th>Institution_Group</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td> Óscar Alberto Zapata Noreña</td>
      <td> GFIF</td>
    </tr>
  </tbody>
</table>

In [22]:
a=articles('citations.csv',user='-6mndWkAAAAJ',authors_file='authors.csv',group_file='groups.csv')

In [23]:
fulldoi=a.articles_update()

Updating entry:0.1.2.

In [24]:
a.articles

,Authors,Title,Publication,Volume,Number,Pages,Year,Publisher,Institution_Authors,Institution_Groups,DOI,ISSN,DOI_Journal,Cites,Impact_Factor,Quartil,Journal_Hindex
0,"Sierra, D Aristizabal; Kubo, Jisuke; Suematsu, Daijiro; Restrepo, D; Zapata, Oscar;","Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals",Physical Review D,79,1,13011,2009,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.79.013011,1550-7998;1550-2368,Physical Review D,83,4.643,Q1,259
1,"Restrepo, Diego; Taoso, Marco; Valle, JWF; Zapata, Oscar;",Gravitino dark matter and neutrino masses with bilinear R-parity violation,Physical Review D,85,2,23523,2012,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.85.023523,1550-7998;1550-2368,Physical Review D,40,4.643,Q1,259
2,"Choi, Ki-Young; Restrepo, Diego; Yaguna, Carlos E; Zapata, Oscar;",Indirect detection of gravitino dark matter including its three-body decays,Journal of Cosmology and Astroparticle Physics,2010,10,33,2010,IOP Publishing,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1088/1475-7516/2010/10/033,1475-7516,Journal of Cosmology and Astroparticle Physics,33,5.81,Q3,70


In [25]:
fulldoi.ix[2]

DOI                                                                                                                                                                                           10.1088/1475-7516/2010/10/033
ISSN                                                                                                                                                                                                            [1475-7516]
URL                                                                                                                                                                         http://dx.doi.org/10.1088/1475-7516/2010/10/033
article-number                                                                                                                                                                                                             
author              [{'affiliation': [], 'family': 'Choi', 'given': 'Ki-Young'}, {'affiliation': [], 'family': 'Restrepo

## Update authors/Grupos data bases

In [26]:
a.add_institution_author()

Full name: Diego Alejandro Restrepo Quintero
Author names
(Example: Perez, Juan;Perez;Pérez, J.):
Restrepo, D;Restrepo, Diego
Additional identitication number: 0


In [27]:
a.add_institution_group()

Find author by first name and first surname
(Example: Juan Pérez);
Diego Restrepo
Group name for
Diego Alejandro Restrepo Quintero
(Example: IA: Inteligencia Artificial)GFIF


Save updated data bases:

In [33]:
a.institution_authors.to_csv('authors.csv',index=False)
a.institution_group.to_csv('groups.csv',index=False)

## Save uptated data base

In [34]:
a.to_csv('newcitations.csv')

Restore saved data base

In [35]:
arts=articles('newcitations.csv',user='-6mndWkAAAAJ',authors_file='authors.csv',group_file='groups.csv')

In [36]:
arts.institution_authors

,Author_Names,Control,Full_Name
0,"Zapata, Oscar;Zapata, O.",0,Óscar Alberto Zapata Noreña
1,"Restrepo, D;Restrepo, Diego",0,Diego Alejandro Restrepo Quintero


In [37]:
arts.institution_group

,Full_Name,Institution_Group
0,Óscar Alberto Zapata Noreña,GFIF
1,Diego Alejandro Restrepo Quintero,GFIF


In [38]:
arts.articles_update()

Updating entry:0.1.2.

""


In [39]:
arts.articles

,Authors,Title,Publication,Volume,Number,Pages,Year,Publisher,Institution_Authors,Institution_Groups,DOI,ISSN,DOI_Journal,Cites,Impact_Factor,Quartil,Journal_Hindex
0,"Sierra, D Aristizabal; Kubo, Jisuke; Suematsu, Daijiro; Restrepo, D; Zapata, Oscar;","Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals",Physical Review D,79,1,13011,2009,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.79.013011,1550-7998;1550-2368,Physical Review D,83,4.643,Q1,259
1,"Restrepo, Diego; Taoso, Marco; Valle, JWF; Zapata, Oscar;",Gravitino dark matter and neutrino masses with bilinear R-parity violation,Physical Review D,85,2,23523,2012,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.85.023523,1550-7998;1550-2368,Physical Review D,40,4.643,Q1,259
2,"Choi, Ki-Young; Restrepo, Diego; Yaguna, Carlos E; Zapata, Oscar;",Indirect detection of gravitino dark matter including its three-body decays,Journal of Cosmology and Astroparticle Physics,2010,10,33,2010,IOP Publishing,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1088/1475-7516/2010/10/033,1475-7516,Journal of Cosmology and Astroparticle Physics,33,5.810,Q3,70


In [11]:
import os
os.path.isfile('authors.csv')

True

In [12]:
self=a

In [15]:
kk=pd.DataFrame()
if kk.shape[0]==0:
    print(1)

1


[Develompent notebook](./dev.ipynb)